##　Ｅｘｃｅｒｃｉｃｅ

In [43]:
import pandas as pd
import numpy as np

In [44]:
df = pd.read_excel('../data/spx_data_weekly.xlsx',sheet_name="spx data", header=[0,1], index_col=0)
df.index = pd.to_datetime(df.index)
valid_tickers = []
min_weeks = 52 * 5
for ticker in df.columns.levels[0]:
    if ticker not in df.columns: 
        continue
    if df[ticker].dropna().shape[0] >= min_weeks:
        valid_tickers.append(ticker)

df_filtered = df[valid_tickers]
df.head(), df_filtered.shape

(                  A                              AAPL                  \
             PX_LAST EQY_DVD_YLD_IND PE_RATIO  PX_LAST EQY_DVD_YLD_IND   
 date                                                                    
 2015-07-03  36.4044          1.0988  27.4720  28.2589          1.8401   
 2015-07-10  36.2388          1.1038  27.3471  27.5527          1.8873   
 2015-07-17  36.7447          1.0886  27.7288  28.9696          1.7950   
 2015-07-24  36.1560          1.1063  27.2846  27.8253          1.8688   
 2015-07-31  37.6645          1.0620  26.6192  27.1101          1.9181   
 
                         ABBV                             ABNB  ...      YUM  \
            PE_RATIO  PX_LAST EQY_DVD_YLD_IND PE_RATIO PX_LAST  ... PE_RATIO   
 date                                                           ...            
 2015-07-03  13.0592  44.2648          4.6086  11.9239     NaN  ...  18.6689   
 2015-07-10  12.7328  44.9267          4.5407  12.1022     NaN  ...  18.5282   
 2015-

In [45]:
# 1.1
def get_dividend_yield_extremes(df):
    div_yld = df.xs("EQY_DVD_YLD_IND", axis=1, level=1)
    max_yield = div_yld.idxmax(axis=1)  
    min_yield = div_yld.idxmin(axis=1) 
    return pd.DataFrame({'max_yield_ticker': max_yield, 'min_yield_ticker': min_yield})

extremes_df = get_dividend_yield_extremes(df_filtered)
print(extremes_df.tail())

           max_yield_ticker min_yield_ticker
date                                        
2025-05-30              DOW             NVDA
2025-06-06              DOW             NVDA
2025-06-13              DOW             NVDA
2025-06-20              DOW             NVDA
2025-06-27              DOW             NVDA


In [46]:
def get_average_dividend_yield(df, n_weeks=52):
    div_yld = df.xs("EQY_DVD_YLD_IND", axis=1, level=1).tail(n_weeks)
    mean_yield = div_yld.mean()
    max_ticker = mean_yield.idxmax()
    min_ticker = mean_yield.idxmin()
    return max_ticker, mean_yield[max_ticker], min_ticker, mean_yield[min_ticker]

high, high_val, low, low_val = get_average_dividend_yield(df_filtered)
print(f"Highest average dividend ratio：{high}（{high_val:.2f}%）")
print(f"Lowest average dividend ratio：{low}（{low_val:.2f}%）")

Highest average dividend ratio：MO（7.95%）
Lowest average dividend ratio：NVDA（0.03%）


<span style="color: blue;">

ANS: 

Over the past year, MO had the highest average dividend yield at 7.95%, while NVDA had the lowest at just 0.03%.

The elevated yield for MO was primarily due to its consistently high dividend payouts and relatively stable or modestly declining stock price, indicating a strong commitment to income-focused investors. In contrast, NVDA's extremely low yield reflects its minimal dividend distribution and rapidly rising stock price, characteristic of a high-growth company that reinvests earnings rather than returning capital to shareholders.

Overall, the variation in yields appears to be driven more by differences in dividend policy (D) than by changes in price (P), especially in the case of NVDA.

In [47]:
# 1.2

# --- Extract dividend yield and price ---
dvd_yld = df_filtered.xs("EQY_DVD_YLD_IND", axis=1, level=1)
price = df_filtered.xs("PX_LAST", axis=1, level=1)

# --- Weekly returns ---
ret = price.pct_change().shift(-1)

# --- Ranking each week ---
rank = dvd_yld.rank(axis=1, ascending=False)

n = rank.shape[1]
top_n = int(np.floor(0.2 * n))

# --- Long-only weights (equal 0.01) ---
weights_long = (rank <= top_n).astype(float) * 0.01

# --- Portfolio return ---
carry_ret = (weights_long * ret).sum(axis=1)
carry_ret = carry_ret.dropna()
carry_ret.head()




date
2015-07-03    0.005380
2015-07-10    0.004836
2015-07-17   -0.018145
2015-07-24    0.017742
2015-07-31   -0.005158
dtype: float64

In [48]:
# 1.3
# --- Long-Short weights ---
weights_ls = pd.DataFrame(0, index=rank.index, columns=rank.columns)

weights_ls[rank <= top_n] = 0.01
weights_ls[rank > (n - top_n)] = -0.01

# --- Long-short portfolio return ---
ls_ret = (weights_ls * ret).sum(axis=1)
ls_ret = ls_ret.dropna()
ls_ret.head()

C:\Users\User\AppData\Local\Temp\ipykernel_2288\1727887067.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  weights_ls[rank <= top_n] = 0.01
C:\Users\User\AppData\Local\Temp\ipykernel_2288\1727887067.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  weights_ls[rank > (n - top_n)] = -0.01


date
2015-07-03    0.005102
2015-07-10    0.000766
2015-07-17   -0.009259
2015-07-24    0.009111
2015-07-31   -0.002786
dtype: float64

In [49]:
# 1.4

def perf_stats(r):
    ann_mean = r.mean() * 52
    ann_vol = r.std() * np.sqrt(52)
    sharpe = ann_mean / ann_vol
    
    skew = r.skew()
    var_5 = r.quantile(0.05)
    cvar_5 = r[r <= var_5].mean()
    
    cum = (1 + r).cumprod()
    dd = cum / cum.cummax() - 1
    max_dd = dd.min()
    
    return pd.Series({
        "Mean": ann_mean,
        "Vol": ann_vol,
        "Sharpe": sharpe,
        "Skew": skew,
        "VaR_5%": var_5,
        "CVaR_5%": cvar_5,
        "MaxDD": max_dd
    })

df_add = pd.read_excel(
    '../data/spx_data_weekly.xlsx',
    sheet_name='additional data',
    header=[0,1],
    index_col=0
)

df_add.index = pd.to_datetime(df_add.index)
spy_price = df_add.xs("SPY", axis=1, level=0).xs("PX_LAST", axis=1)
spy_ret = spy_price.pct_change().shift(-1).dropna()
common_idx = carry_ret.index.intersection(spy_ret.index)

carry_ret = carry_ret.loc[common_idx]
ls_ret = ls_ret.loc[common_idx]
spy_ret = spy_ret.loc[common_idx]

perf_table = pd.concat([
    perf_stats(carry_ret),
    perf_stats(ls_ret),
    perf_stats(spy_ret)
], axis=1)

perf_table.columns = ["Carry Long", "Carry Long-Short", "SPY"]
print(perf_table)

         Carry Long  Carry Long-Short       SPY
Mean       0.112670          0.021356  0.140826
Vol        0.160823          0.095895  0.173193
Sharpe     0.700582          0.222706  0.813116
Skew       0.285957          1.559453 -0.595027
VaR_5%    -0.028205         -0.018282 -0.033571
CVaR_5%   -0.049354         -0.026737 -0.056766
MaxDD     -0.347861         -0.179768 -0.318291


In [50]:
#2

import pandas as pd
import numpy as np
import statsmodels.api as sm

def run_lfd(Y, X, freq=52):
    X = sm.add_constant(X)
    model = sm.OLS(Y, X, missing="drop").fit()

    alpha_ann = model.params["const"] * freq
    beta = model.params.drop("const")
    r2 = model.rsquared

    return alpha_ann, beta, r2, model


In [51]:

sector_px = pd.read_excel(
    "../data/spx_data_weekly.xlsx",   # ← 你的 sector excel
    sheet_name="sector data",
    header=[0,1],
    index_col=0,
    parse_dates=True
)

sector_px.index.name = "date"


In [52]:
valid_sectors = [
    'XLK','XLI','XLF','XLC','XLRE','XLE','XLY',
    'XLB','XLV','XLU','XLP'
]

sector_price = sector_px.loc[:, (valid_sectors, "PX_LAST")]

sector_ret = sector_price.pct_change()
sector_ret.columns = sector_ret.columns.get_level_values(0)

# 與策略報酬對齊
sector_ret = sector_ret.loc[common_idx]
sector_ret.head()

,XLK,XLI,XLF,XLC,XLRE,XLE,XLY,XLB,XLV,XLU,XLP
date,,,,,,,,,,,
2015-07-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-10,-0.007187,-0.001108,0.000000,NaN,NaN,-0.013954,0.006087,-0.016698,0.005879,0.017523,0.020965
2015-07-17,0.048262,0.013846,0.030434,NaN,NaN,-0.014288,0.019056,-0.000839,0.022450,0.008842,0.018099
2015-07-24,-0.024863,-0.036781,-0.010633,NaN,NaN,-0.040447,-0.003790,-0.054133,-0.027543,-0.023529,-0.007789
2015-07-31,0.005194,0.024952,0.003586,NaN,NaN,-0.001870,0.017374,0.019298,0.023246,0.039216,0.012681


In [53]:
# 2.1

alpha_lo, beta_lo, r2_lo, m_lo = run_lfd(carry_ret, spy_ret.to_frame("SPY"))
alpha_ls, beta_ls, r2_ls, m_ls = run_lfd(ls_ret, spy_ret.to_frame("SPY"))

market_summary = pd.DataFrame({
    "LO": [alpha_lo, beta_lo.iloc[0], r2_lo],
    "LS": [alpha_ls, beta_ls.iloc[0], r2_ls]
}, index=["Alpha (ann.)", "Beta vs SPY", "R²"])

market_summary


,LO,LS
Alpha (ann.),0.006958,0.011508
Beta vs SPY,0.750660,0.069934
R²,0.653506,0.015953


In [54]:
corr_df = pd.concat([
    carry_ret.rename("LO"),
    ls_ret.rename("LS"),
    spy_ret.rename("SPY")
], axis=1)

corr_matrix = corr_df.corr()
corr_matrix


,LO,LS,SPY
LO,1.000000,0.643840,0.808397
LS,0.643840,1.000000,0.126306
SPY,0.808397,0.126306,1.000000


<span style="color: blue;">

ANS:

LO and LS strategies compare to SPY don't have strong alpha.LO has a high beta while LS has a low beta, which is not suprising.

In [55]:
# 2.2
additional_px = pd.read_excel(
    "../data/spx_data_weekly.xlsx",
    sheet_name="additional data",
    header=[0,1],
    index_col=0,
    parse_dates=True
)

additional_px.index.name = "date"
additional_px.head()

,SPY,IEF,IYR,GLD,USO,XBTUSD BGN Curncy,SHV
,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST
date,,,,,,,
2015-07-03,174.2734,83.7708,52.7722,111.76,151.44,255.22,90.0731
2015-07-10,174.4163,83.7468,53.6596,111.49,141.36,287.13,90.0731
2015-07-17,178.6195,84.0830,54.1105,108.65,135.92,278.27,90.0812
2015-07-24,174.8534,84.6752,53.7905,105.35,128.24,288.16,90.0731
2015-07-31,176.9550,85.1874,54.4306,104.93,124.72,284.69,90.0567


In [56]:
add_price = additional_px.xs("PX_LAST", axis=1, level=1)
factor_price = add_price.drop(columns=["SPY", "SHV"], errors="ignore")
factor_ret = factor_price.pct_change()
factor_ret = factor_ret.loc[common_idx]
factor_ret.head()

,IEF,IYR,GLD,USO,XBTUSD BGN Curncy
date,,,,,
2015-07-03,NaN,NaN,NaN,NaN,NaN
2015-07-10,-0.000286,0.016816,-0.002416,-0.066561,0.125029
2015-07-17,0.004014,0.008403,-0.025473,-0.038483,-0.030857
2015-07-24,0.007043,-0.005914,-0.030373,-0.056504,0.035541
2015-07-31,0.006049,0.011900,-0.003987,-0.027449,-0.012042


In [57]:
alpha_lo_sec, beta_lo_sec, r2_lo_sec, m_lo_sec = run_lfd(
    carry_ret, factor_ret
)

alpha_ls_sec, beta_ls_sec, r2_ls_sec, m_ls_sec = run_lfd(
    ls_ret, factor_ret
)

factor_beta = pd.DataFrame({
    "LO": beta_lo_sec,
    "LS": beta_ls_sec
})

factor_beta


,LO,LS
IEF,0.301775,0.007604
IYR,-0.159863,-0.058450
GLD,-0.041631,-0.016525
USO,0.047575,0.045882
XBTUSD BGN Curncy,0.026360,-0.002960


In [58]:
factor_stats = pd.DataFrame({
    "Alpha (ann.)": [alpha_lo_sec, alpha_ls_sec],
    "R²": [r2_lo_sec, r2_ls_sec]
}, index=["LO", "LS"])

factor_stats


,Alpha (ann.),R²
LO,0.103318,0.061621
LS,0.028947,0.045882


In [59]:
factor_std = factor_ret.std()

std_exposure = pd.DataFrame({
    "LO": beta_lo_sec * factor_std,
    "LS": beta_ls_sec * factor_std
})

max_lo_sector = std_exposure["LO"].abs().idxmax()
max_ls_sector = std_exposure["LS"].abs().idxmax()

std_exposure, max_lo_sector, max_ls_sector


(                         LO        LS
 IEF                0.002699  0.000068
 IYR               -0.004857 -0.001776
 GLD               -0.000826 -0.000328
 USO                0.002601  0.002508
 XBTUSD BGN Curncy  0.002502 -0.000281,
 'IYR',
 'USO')

<span style="color: blue;">
ANS:

The carry strategy is both neutral to sectors. IYR has the largest exposure, thought it is still small.

In [60]:
mag7 = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA']
mag_px = df.loc[:, (mag7, "PX_LAST")]
mag_ret = mag_px.pct_change().mean(axis=1)
mag_ret = mag_ret.loc[common_idx]
mag_ret.name = "MAG"
mag_ret.head()


date
2015-07-03         NaN
2015-07-10   -0.012956
2015-07-17    0.085624
2015-07-24   -0.009611
2015-07-31    0.001059
Name: MAG, dtype: float64

In [61]:
X_mag = pd.concat([spy_ret, mag_ret], axis=1)
X_mag.columns = ["SPY", "MAG"]

alpha_lo_mag, beta_lo_mag, r2_lo_mag, m_lo_mag = run_lfd(
    carry_ret, X_mag
)

alpha_ls_mag, beta_ls_mag, r2_ls_mag, m_ls_mag = run_lfd(
    ls_ret, X_mag
)

mag_summary = pd.DataFrame({
    "LO Alpha (ann.)": [alpha_lo_mag],
    "LO β_SPY": [beta_lo_mag["SPY"]],
    "LO β_MAG": [beta_lo_mag["MAG"]],
    "LO R²": [r2_lo_mag],
    
    "LS Alpha (ann.)": [alpha_ls_mag],
    "LS β_SPY": [beta_ls_mag["SPY"]],
    "LS β_MAG": [beta_ls_mag["MAG"]],
    "LS R²": [r2_ls_mag]
})

mag_summary



,LO Alpha (ann.),LO β_SPY,LO β_MAG,LO R²,LS Alpha (ann.),LS β_SPY,LS β_MAG,LS R²
0,0.002618,0.751736,0.010803,0.653896,0.00791,0.070806,0.00868,0.016553


<span style="color: blue;">

ANS:

For the LO strategy, the SPY beta is 0.75 and the MAG beta is near zero (0.01), with a high R² of 0.65. This shows that the LO strategy is mainly driven by overall market exposure, with negligible direct exposure to the Magnificent Seven.

For the LS strategy, both SPY and MAG betas are close to zero, and the R² is only 0.02, indicating that the LS strategy is largely independent of both market and MAG factors. Its return is therefore primarily driven by non-market sources.

Overall, adding the MAG factor does not materially improve the explanatory power of the model.


In [62]:
# 3

window = 52 * 5

hedge_beta_lo = pd.DataFrame(index=sector_ret.index, columns=sector_ret.columns)

for t in range(window, len(sector_ret)):
    Y = carry_ret.iloc[t-window:t]
    X = sm.add_constant(sector_ret.iloc[t-window:t])
    model = sm.OLS(Y, X, missing="drop").fit()
    hedge_beta_lo.iloc[t] = model.params.drop("const")

hedge_beta_ls = pd.DataFrame(index=sector_ret.index, columns=sector_ret.columns)

for t in range(window, len(sector_ret)):
    Y = ls_ret.iloc[t-window:t]
    X = sm.add_constant(sector_ret.iloc[t-window:t])
    model = sm.OLS(Y, X, missing="drop").fit()
    hedge_beta_ls.iloc[t] = model.params.drop("const")



In [63]:
hedge_beta_lo = hedge_beta_lo.shift(1)
hedge_beta_ls = hedge_beta_ls.shift(1)

hedged_lo_ret = carry_ret - (hedge_beta_lo * sector_ret).sum(axis=1)
hedged_lo_ret = hedged_lo_ret.dropna()

hedged_ls_ret = ls_ret - (hedge_beta_ls * sector_ret).sum(axis=1)
hedged_ls_ret = hedged_ls_ret.dropna()



In [64]:
# 3.1

def univariate_stats(r, freq=52):
    return pd.Series({
        "Mean (ann.)": r.mean() * freq,
        "Vol (ann.)": r.std() * np.sqrt(freq),
        "Sharpe": (r.mean() / r.std()) * np.sqrt(freq),
        "Min": r.min(),
        "Max": r.max()
    })

stats_hedged = pd.concat([
    univariate_stats(hedged_lo_ret).rename("Hedged LO"),
    univariate_stats(hedged_ls_ret).rename("Hedged LS")
], axis=1)

stats_hedged


,Hedged LO,Hedged LS
Mean (ann.),0.112963,0.021268
Vol (ann.),0.172117,0.099590
Sharpe,0.656313,0.213550
Min,-0.145055,-0.051743
Max,0.185052,0.106062


In [65]:
# 3.2

common_idx_hedged_lo = hedged_lo_ret.index.intersection(factor_ret.index)
common_idx_hedged_ls = hedged_ls_ret.index.intersection(factor_ret.index)

Y_lo = hedged_lo_ret.loc[common_idx_hedged_lo]
X_lo = factor_ret.loc[common_idx_hedged_lo]
Y_lo = pd.to_numeric(Y_lo, errors="coerce")

alpha_hlo, beta_hlo, r2_hlo, m_hlo = run_lfd(Y_lo, X_lo)

Y_ls = hedged_ls_ret.loc[common_idx_hedged_ls]
X_ls = factor_ret.loc[common_idx_hedged_ls]
Y_ls = pd.to_numeric(Y_ls, errors="coerce")

alpha_hls, beta_hls, r2_hls, m_hls = run_lfd(Y_ls, X_ls)

hedged_factor_beta = pd.DataFrame({
    "Hedged LO": beta_hlo,
    "Hedged LS": beta_hls
})

hedged_factor_stats = pd.DataFrame({
    "Alpha (ann.)": [alpha_hlo, alpha_hls],
    "R²": [r2_hlo, r2_hls]
}, index=["Hedged LO", "Hedged LS"])


hedged_factor_stats



,Alpha (ann.),R²
Hedged LO,0.099939,0.03817
Hedged LS,0.024913,0.02200


In [66]:
hedged_factor_beta

,Hedged LO,Hedged LS
IEF,0.435167,0.085176
IYR,-0.097569,-0.033827
GLD,-0.048955,-0.015836
USO,0.028919,0.037118
XBTUSD BGN Curncy,0.024076,-0.001699


In [67]:
# 4



In [68]:
# 4.1

forecast = pd.DataFrame(index=dvd_yld.index, columns=dvd_yld.columns)

for t in dvd_yld.index[:-1]:  
    y = dvd_yld.loc[t].dropna()
    q20 = y.quantile(0.2)
    q80 = y.quantile(0.8)

    long_names  = y[y >= q80].index
    short_names = y[y <= q20].index

    forecast.loc[t, long_names]  = 0.001   # +0.1%
    forecast.loc[t, short_names] = -0.001  # -0.1%

realized = ret.shift(-1).loc[forecast.index]

f_vec = forecast.stack()
r_vec = realized.stack()

data_41 = pd.concat([f_vec, r_vec], axis=1).dropna()
data_41.columns = ["forecast", "realized"]

from sklearn.metrics import r2_score

r2_41 = r2_score(data_41["realized"], data_41["forecast"])
corr_41 = data_41["forecast"].corr(data_41["realized"])

r2_41, corr_41



(-0.005402906220958847, np.float64(-0.0033968384077423933))

In [ ]:
# 4.2
window = 52 * 5  # 5 years weekly

cov = ret.rolling(window).cov(spy_ret)
var = spy_ret.rolling(window).var()

beta_spy = cov.div(var, axis=0)

resid_spy_fast = ret.shift(-1) - beta_spy.shift(1).multiply(
    spy_ret.shift(-1), axis=0
)

realized_hedged_41 = resid_spy_fast.loc[forecast.index]

fh_vec = forecast.stack()
rh_vec = realized_hedged_41.stack()

data_42 = pd.concat([fh_vec, rh_vec], axis=1).dropna()
data_42.columns = ["forecast", "hedged_realized"]


r2_42 = r2_score(data_42["hedged_realized"], data_42["forecast"])
corr_42 = data_42["forecast"].corr(data_42["hedged_realized"])

r2_42, corr_42



(-0.0005576380030560824, np.float64(0.004266412960789664))

In [92]:
# 4.3


# --- Data Initialization (Assuming these DataFrames are already loaded) ---
# ret: Asset returns (Time, Assets)
# sector_ret: Sector returns (Time, Sectors) - LFDs
# forecast: Forecasted asset returns (Time, Assets)

# --- 1. Data Cleaning and Alignment ---

# Find the common index across all necessary DataFrames
common_index = ret.index.intersection(sector_ret.index).intersection(forecast.index)

# Reindex and align the data
sector_ret_aligned = sector_ret.reindex(common_index)
ret_aligned = ret.reindex(common_index)
forecast_aligned = forecast.reindex(common_index)

# Drop rows where factors or asset returns are missing, ensuring clean windows
# We only use indices where both asset returns and sector returns are available
valid_data_index = pd.concat(
    [sector_ret_aligned, ret_aligned], axis=1
).dropna(how='any').index

sector_ret_clean = sector_ret_aligned.loc[valid_data_index]
ret_clean = ret_aligned.loc[valid_data_index]
forecast_clean = forecast_aligned.loc[valid_data_index]

window = 52 * 5  # 5 years weekly

# --- 2. Rolling Multiple Regression (OLS) for Beta Calculation ---

asset_betas = {} # Stores (Time, Sectors) Beta matrix for each asset

for asset in ret_clean.columns:
    betas_list = []
    
    # Iterate through the rolling window
    for end_idx in range(window, len(ret_clean) + 1):
        window_slice = ret_clean.index[end_idx - window:end_idx]
        
        # Dependent variable (Asset return)
        Y_window = ret_clean.loc[window_slice, asset].values
        # Independent variables (Sector returns)
        X_window = sector_ret_clean.loc[window_slice].values
        
        # Add intercept column for OLS
        X_with_intercept = np.column_stack([np.ones(window), X_window])
        
        # Solve for Beta coefficients using Least Squares (OLS)
        # coeffs = [Intercept, Beta_sector1, Beta_sector2, ...]
        coeffs, residuals, rank, s = np.linalg.lstsq(X_with_intercept, Y_window, rcond=None)
        
        # Store only the Beta coefficients (excluding the Intercept)
        betas_list.append(coeffs[1:])
        
    # Convert results to DataFrame, indexed by the end of the window (time t)
    betas_df = pd.DataFrame(
        betas_list,
        index=ret_clean.index[window - 1:],
        columns=sector_ret_clean.columns
    )
    asset_betas[asset] = betas_df

# --- 3. Calculating Hedged Realized Returns (Residuals) ---

# Initialize DataFrame for residuals (hedged returns)
resid_sector_fast = pd.DataFrame(index=ret_clean.index, columns=ret_clean.columns)

# R_i, t+1 - sum(Beta_i, j, t * F_j, t+1)
for asset in ret_clean.columns:
    if asset in asset_betas:
        # Beta_t (Shifted forward by 1 for alignment)
        beta_asset_t_shifted = asset_betas[asset].shift(1)
        
        # Factor_t+1 (Shifted backward by 1 for alignment)
        sector_ret_tplus1_shifted = sector_ret_clean.shift(-1)
        
        # Calculate the hedging return: Beta_t * Factor_t+1 (dot product across sectors)
        # Using element-wise multiplication (*) followed by summation (sum(axis=1))
        hedging_return = (beta_asset_t_shifted * sector_ret_tplus1_shifted).sum(axis=1)

        # Asset_t+1 (Shifted backward by 1 for alignment)
        ret_tplus1_shifted = ret_clean[asset].shift(-1)
        
        # Residual = Realized_Return - Hedging_Return
        resid_sector_fast[asset] = ret_tplus1_shifted - hedging_return

# Final realized hedged data, aligned with the forecast index
realized_hedged_43 = resid_sector_fast.loc[forecast_clean.index]

# --- 4. Performance Metrics (R2 and Correlation) ---

# Stack into vectors
fh_vec = forecast_clean.stack()
rh_vec = realized_hedged_43.stack()

# Combine and drop NaNs for final comparison
data_43 = pd.concat([fh_vec, rh_vec], axis=1).dropna()
data_43.columns = ["forecast", "hedged_realized"]

# Calculate R2 and Correlation
r2_43 = r2_score(data_43["hedged_realized"], data_43["forecast"])
corr_43 = data_43["forecast"].corr(data_43["hedged_realized"])

# Output the results
r2_43, corr_43



(-0.008987698147000955, np.float64(0.0012504744850152499))

In [93]:
forecast_summary = pd.DataFrame({
    "R²": [r2_41, r2_42, r2_43],
    "Correlation": [corr_41, corr_42, corr_43]
}, index=[
    "4.1 Raw Returns",
    "4.2 SPY-Hedged",
    "4.3 Sector-Hedged"
])

forecast_summary


,R²,Correlation
4.1 Raw Returns,-0.005403,-0.003397
4.2 SPY-Hedged,-0.000558,0.004266
4.3 Sector-Hedged,-0.008988,0.001250


In [94]:
# 5

from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [95]:
target_ticker = "AAPL" 

y = ret[target_ticker].dropna()

X = ret.drop(columns=[target_ticker]).loc[y.index]
df_xy = pd.concat([y, X], axis=1).dropna()
y = df_xy[target_ticker]
X = df_xy.drop(columns=[target_ticker])
print("LASSO replication sample size:", X.shape)

lasso_model = make_pipeline(
    StandardScaler(with_mean=True, with_std=True),
    LassoCV(
        cv=5,
        alphas=np.logspace(-4, 0, 30),
        random_state=0,
        n_jobs=-1,
    )
)

lasso_model.fit(X, y)

lasso = lasso_model[-1]
weights = pd.Series(lasso.coef_, index=X.columns)

weights_nonzero = weights[weights.abs() > 1e-4].sort_values(ascending=False)
print("Selected replicating assets (non-zero weights):")
print(weights_nonzero.head(20))

rep_ret = X[weights_nonzero.index].dot(weights_nonzero)

common_idx = y.index.intersection(rep_ret.index)
y_real = y.loc[common_idx]
rep_real = rep_ret.loc[common_idx]

rep_r2 = r2_score(y_real, rep_real)
rep_corr = y_real.corr(rep_real)

rep_stats = pd.Series({
    "R² (realized returns)": rep_r2,
    "Corr (realized returns)": rep_corr,
    "RMSE": np.sqrt(((y_real - rep_real) ** 2).mean())
})

print("\nReplication quality stats (realized returns):")
print(rep_stats)



LASSO replication sample size: (275, 378)


c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.383e-05, tolerance: 3.689e-05
  model = cd_fast.enet_coordinate_descent(
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.634e-05, tolerance: 3.237e-05
  model = cd_fast.enet_coordinate_descent(
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale 

Selected replicating assets (non-zero weights):
QCOM    0.008957
MSFT    0.007157
SWKS    0.004394
INTU    0.003538
COST    0.002636
ROP     0.002621
CHRW    0.002483
VRSK    0.001211
CHD     0.001033
MCD     0.000921
V       0.000741
HPQ     0.000608
CLX     0.000375
EFX     0.000164
CAT    -0.000536
dtype: float64

Replication quality stats (realized returns):
R² (realized returns)      0.024363
Corr (realized returns)    0.778035
RMSE                       0.038777
dtype: float64


c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.998e-04, tolerance: 3.687e-05
  model = cd_fast.enet_coordinate_descent(
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.449e-04, tolerance: 3.015e-05
  model = cd_fast.enet_coordinate_descent(
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale 

In [96]:
# 5.1

forecast_aligned = forecast.reindex(index=common_idx)

forecast_filled = forecast_aligned.fillna(0.0)

if target_ticker not in forecast_filled.columns:
    raise KeyError(f"{target_ticker} not found in forecast columns.")

target_forecast = forecast_filled[target_ticker]

rep_forecast = (forecast_filled[weights_nonzero.index]
                .mul(weights_nonzero, axis=1)
                .sum(axis=1))

mask = target_forecast.notna() & rep_forecast.notna()
target_forecast_use = target_forecast[mask]
rep_forecast_use = rep_forecast[mask]

fc_r2 = r2_score(target_forecast_use, rep_forecast_use)
fc_corr = target_forecast_use.corr(rep_forecast_use)

forecast_consistency_stats = pd.Series({
    "R² (forecast target vs replica)": fc_r2,
    "Corr (forecast target vs replica)": fc_corr,
    "Mean diff (target - replica)": (target_forecast_use - rep_forecast_use).mean(),
    "Std diff": (target_forecast_use - rep_forecast_use).std()
})

print("\nForecast consistency stats:")
print(forecast_consistency_stats)


Forecast consistency stats:
R² (forecast target vs replica)      0.000000
Corr (forecast target vs replica)         NaN
Mean diff (target - replica)        -0.000983
Std diff                             0.000002
dtype: float64


C:\Users\User\AppData\Local\Temp\ipykernel_2288\3341040685.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  forecast_filled = forecast_aligned.fillna(0.0)
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [97]:
# 5.2

diff = target_forecast_use - rep_forecast_use
t_stat = diff.mean() / (diff.std(ddof=1) / np.sqrt(len(diff)))

arb_stats = pd.Series({
    "Mean forecast diff (target - replica)": diff.mean(),
    "Std forecast diff": diff.std(ddof=1),
    "t-stat of diff (≈ arbitrage signal strength)": t_stat,
    "Number of periods": len(diff)
})

print("\nArbitrage implication stats:")
print(arb_stats)


Arbitrage implication stats:
Mean forecast diff (target - replica)             -0.000983
Std forecast diff                                  0.000002
t-stat of diff (≈ arbitrage signal strength)   -6765.391834
Number of periods                                275.000000
dtype: float64


<span style="color: blue;">

ANS:

The replication portfolio exhibits strong comovement with the target asset in realized returns, but the R² remains low, indicating that the target is not perfectly spanned by other assets. Moreover, due to the discrete nature of our dividend-yield-based forecasts, the forecast consistency test becomes degenerate. Therefore, while small systematic differences between the target and replication forecasts are observed, they do not constitute evidence of an economically meaningful arbitrage opportunity.